In [824]:
import math
from json import JSONEncoder

class depotEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__

depot_name = "Song Than depot"
outfile = "./tbd.json"

In [825]:

class Point:
    def __init__(self,x,y) -> None:
        self.x = x
        self.y = y
    
    def translate(self, dx=0, dy=0):
        self.x += dx
        self.y += dy

    def scale(self, cox=1, coy=1):
        self.x *= cox
        self.y *= coy

class Shape:
    def __init__(self, seq, vis=1) -> None:
        self.seq = seq
        self.length = len(seq)
        self.visible = vis
    
    def scale(self,cox=1, coy=1):
        for i in range(self.length):
            self.seq[i].scale(cox,coy)
    
    def translate(self,dx=0, dy=0):
        for i in range(self.length):
            self.seq[i].translate(dx,dy)

class Container: 
    def __init__(self, contID, contSize, HangTau, Block, Bay, Row, Tier):
        self.ContID = contID
        self.ContTypeSizeID = contSize
        self.HangTauID = HangTau
        self.Block = Block
        self.Bay = Bay
        self.Row = Row
        self.Tier = Tier
        self.angle = 0
        self.x=0
        self.y=0
        self.z=0

class Text: 
    def __init__(self, content, x,y,size=8, angle=0) -> None:
        self.content = content
        self.position = Point(x,y)
        self.size = size
        self.angle = angle


In [826]:
# read text
def read_text(path):
    result = []
    fi = open(path, "r")
    str = fi.read()
    a = 0
    p = str.find("<text",a)
    while p>0:
        x = []
        y = []
        b = str.find("/text>",p)
        k = str[p+27:b]
        # print(k)
        text = k[k.find('">')+2:k.find("</")]
        
        k = k[0:k.find("rotate")]
        obr = k.find("(")
        cbr = k.find(")")
        translate = k[obr+1:cbr]
        # print(translate)

        sp = translate.find(" ")
        x = translate[0:sp]
        y = translate[sp+1: len(translate)]

        result.append(Text(text, int(x),int(y)))

        # print("x", x, "y", y)
        a = b
        p = str.find("<text",a)
    return result

def svg2layout(path):
    file = open(path, "r")
    str = file.read()
    layout_array = []
    a = 0
    p = str.find("<polygon",a)
    while p>0:
        x = []
        y = []
        b = str.find("/>",p)
        div = str[p+27:b-1] 
        # print("shape:", div)
        chr_id=0
        type = 0
        sp = div.find(" ",chr_id)
        while sp>0:
            if type:
                # print("y:",div[chr_id:sp])
                y.append(int(div[chr_id:sp]))
            else:
                # print("x:",div[chr_id:sp])
                x.append(int(div[chr_id:sp]))

            type = 1-type
            chr_id = sp
            sp = div.find(" ", sp+1)
        # print("y:",div[chr_id:len(div)])
        y.append(int(div[chr_id:len(div)]))

        a = b
        p = str.find("<polygon",a)
        # print(x)
        # print(y)
        seq = []
        for i in range(len(x)):
            seq.append(Point(x[i],y[i]))
        layout_array.append(Shape(seq))
    return layout_array

# read rect
def read_rect(path):
    result = []
    fi = open(path, "r")
    str = fi.read()
    a = 0
    p = str.find("<rect",a)
    while p>0:
        # x = []
        # y = []
        b = str.find("/>",p)
        k = str[p+5:b]
        print(k)
        px = k.find('x="');
        pxe = k.find('" ', px)
        x = k[px+3:pxe]
        print(x)

        py = k.find('y="')
        pye =k.find('" ', py)
        y = k[py+3:pye]
        print(y)

        pw = k.find('width="')
        pwe =k.find('" ', pw)
        w = k[pw+7:pwe]
        # print(w)

        ph = k.find('height="')
        phe =k.find('" ', ph)
        h = k[ph+8:phe]
        # print(h)

        a = b
        p = str.find("<rect",a)
    return result

In [827]:
class Area:
    def __init__(self, name, x, y, offsetX=0, offsetY=0, angle=0, x_flip=False, y_flip=False, shapeID=None, num_of_bay=15, num_of_row=6, num_of_tier=6) -> None:
        self.name = name
        self.num_of_bay = num_of_bay
        self.num_of_row = num_of_row
        self.num_of_tier = num_of_tier
        self.x_coor = x
        self.y_coor = y
        self.shapeID = shapeID
        self.angle = angle
        self.offset = Point(offsetX, offsetY)
        self.x_flip = x_flip
        self.y_flip = y_flip

class Layout:
    def __init__(self,name,svg_path, text=[]) -> None:
        self.name = name
        self.shape = svg2layout(svg_path)
        self.text = read_text(svg_path)

    def scale(self,cox=1, coy=1):
        for i in range(len(self.shape)):
            self.shape[i].scale(cox,coy)
        for i in range(len(self.text)):
            self.text[i].position.scale(cox, coy)
    
    def translate(self,dx=0, dy=0):
        for i in range(len(self.shape)):
            self.shape[i].translate(dx,dy)
        for i in range(len(self.text)):
            self.text[i].position.translate(dx, dy)

class Ground:
    def __init__(self, shapeID, id1, offsetX=0, offsetY=0, offsetZ=0, angle=0, height=None, width=None, button = None):
        self.shapeID = shapeID
        self.id1 = id1
        self.offsetX = offsetX
        self.offsetY = offsetY
        self.offsetZ = offsetZ
        self.angle = angle
        self.button = button
        self.hei = height
        self.wid = width
class House:
    def __init__(self, type, name, shape, id1=0, id2=2, height=100, angle=0, offsetX=0, offsetY=0) -> None:
        self.type = type
        self.name = name
        self.shape = shape
        self.id1 = id1
        self.id2 = id2
        self.height = height
        self.angle = angle
        self.offsetX = offsetX
        self.offsetY = offsetY

class Depot:
    def __init__(self, name, layout, ground=[], slope=[], area=[], house=[], container_scale=1, num_of_ground_level=1, offset=Point(0,0), scale=1) -> None:        
        self.name = name
        self.layout = layout
        self.contWidth = 8 * container_scale
        self.contHeight = 8.5 * container_scale
        self.contLength = 20 * container_scale
        self.contHalfLength = 10 * container_scale
        self.contGap = math.floor(self.contLength*0.06)
        self.fontSize = 3*container_scale   
        self.Area = area
        self.house = house
        self.ground=ground
        self.offset = offset
        self.scale = scale

class Button:
    def __init__(self, p, text, angle) -> None:
        self.position = p
        self.text = text
        self.angle = angle
        
    def translate(self, x=0, y=0):
        self.position.x +=x;
        self.position.y +=y;

In [828]:
((10.57)/360)*(2*math.pi)

0.18448130193580065

In [829]:
# tbd_layout.shape[5].seq[1].x
-0.8158514559173915 + math.pi/2


0.7549448708775051

In [830]:
# print(tbd_layout.shape[2].seq[3].x)
# print(tbd_layout.shape[2].seq[3].y)

In [831]:
tbd_layout = Layout("General","./tbd.svg")
# tbd_layout.scale(cox=3, coy=3)
# tbd_layout.translate(-400, -1700)

tbd_layout.shape[1].visible = 0.2
tbd_layout.shape[2].visible = 0.2


#button
tbd_layout.shape[14].visible = 0
tbd_layout.shape[15].visible = 0
tbd_layout.shape[16].visible = 0
tbd_layout.shape[17].visible = 0
tbd_layout.shape[18].visible = 0
tbd_layout.shape[19].visible = 0
tbd_layout.shape[20].visible = 0
tbd_layout.shape[21].visible = 0.2
tbd_layout.shape[22].visible = 0.2

btn0 = Button(tbd_layout.shape[14].seq[3], "Area 1a", angle=-1.1257373675363425);
btn1 = Button(tbd_layout.shape[15].seq[3], "Area 1b", angle=-1.0222393428930787);
btn2 = Button(tbd_layout.shape[16].seq[3], "Area 1c", angle=0.18448130193580065);
btn3 = Button(tbd_layout.shape[17].seq[3], "Area 1d", angle=-0.1095892237327239 + math.pi/2);
btn4 = Button(tbd_layout.shape[18].seq[1], "Area 2a", angle=0.05462880558742251);
btn5 = Button(tbd_layout.shape[19].seq[1], "Area 2b", angle=1.6718508904853682);
btn6 = Button(tbd_layout.shape[20].seq[1], "Area 2c", angle=-1.4809118703171884);

ground0 = Ground(1,4, offsetX=tbd_layout.shape[1].seq[3].x, offsetY=tbd_layout.shape[1].seq[3].y, offsetZ=0, angle=0.445058959258554, button=btn0, height=980, width=1060);
ground1 = Ground(1,3, offsetX=tbd_layout.shape[1].seq[3].x-35, offsetY=tbd_layout.shape[1].seq[3].y+25, offsetZ=0, angle=0.5485569839018177, height=1020, width=1060, button=btn1)
ground2 = Ground(1,0, offsetX=tbd_layout.shape[1].seq[3].x, offsetY=tbd_layout.shape[1].seq[3].y, height=820, width=1080, offsetZ=0, angle=0.18448130193580065, button=btn2)
ground3 = Ground(1,0, offsetX=tbd_layout.shape[1].seq[1].x+16, offsetY=tbd_layout.shape[1].seq[1].y-135, height=1040, width=1140, offsetZ=0, angle=-0.1095892237327239, button=btn3)
ground4 = Ground(2,0, offsetX=tbd_layout.shape[2].seq[0].x, offsetY=tbd_layout.shape[2].seq[0].y, height=300, width=700, offsetZ=0, angle=0.05462880558742251, button=btn4)
ground5 = Ground(2,0, offsetX=tbd_layout.shape[2].seq[0].x, offsetY=tbd_layout.shape[2].seq[0].y, height=310, width=700, offsetZ=0, angle=0.1010545636904717, button=btn5)
ground6 = Ground(2,0, offsetX=tbd_layout.shape[2].seq[0].x, offsetY=tbd_layout.shape[2].seq[0].y, height=320, width=700, offsetZ=0, angle=0.08988445647770797, button=btn6)

# add warehouse here
h1 = House(0, "house 1", tbd_layout.shape[3],0,2,200,0.05235987755982988)
h2 = House(0, "house 2", tbd_layout.shape[4],0,2,200,0.05235987755982988)
h3 = House(0, "house 3", tbd_layout.shape[5],0,2,200,0.05235987755982988)
h4 = House(0, "house 4", tbd_layout.shape[6],0,2,200,0.05235987755982988)
h5 = House(0, "house 5", tbd_layout.shape[7],0,2,200,0.05235987755982988)
h6 = House(0, "house 6", tbd_layout.shape[8],0,2,200,0.05235987755982988)
h7 = House(0, "house 7", tbd_layout.shape[9],0,2,200,0.05235987755982988)
h8 = House(0, "house 8", tbd_layout.shape[10],0,2,200,0.05235987755982988)
h9 = House(0, "house 9", tbd_layout.shape[11],0,2,200,0.05235987755982988)
h10 = House(0, "house 10", tbd_layout.shape[12],0,2,200,0.05235987755982988)
h11 = House(0, "house 11", tbd_layout.shape[13],0,2,200,0.05235987755982988)
# ruaCont = House(2, "Rua container", tbd_layout.shape[4],3,1,20, offsetX=34, offsetY=-74)
tbd = Depot("Tây Nam Bình Dương Depot", tbd_layout, ground=[ground0, ground1, ground2, ground3, ground4, ground5, ground6], house=[h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,h11], container_scale=1.8, scale=0.4)
# etd = Depot("etd", tbd_layout, ground=[ground1, ground2], offset=Point(-450,300), house=[wh1, wh2, wh3, ruaCont],container_scale=4.25, scale=0.4)

In [832]:
jn = (depotEncoder().encode(tbd))
ofile = open(outfile, "w")
ofile.write(jn)
ofile.close()